In [32]:
import pandas as pd
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
from stocktrends import Renko


In [33]:
tickers=["ITC.NS"]
ohlcv_data={}
hour_data={}
renko_data = {}

In [34]:
for ticker in tickers:
    temp=yf.download(ticker,period='1mo',interval='5m')
    temp.dropna(how='any',inplace=True)
    ohlcv_data[ticker]=temp
    temp=yf.download(ticker,period='1mo',interval='5m')
    temp.dropna(how='any',inplace=True)
    hour_data[ticker]=temp

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [35]:
def ATR(DF,n=14):
    df=DF.copy()
    print(df)
    df["H-L"]=df["High"]-df["Low"]
    df["H-PC"]=df["High"]-df["Adj Close"].shift(1)
    df["L-PC"]=df["Low"]-df["Adj Close"].shift(1)
    df["TR"]=df[["H-L","H-PC","L-PC"]].max(axis=1,skipna=False)
    df["ATR"]=df["TR"].ewm(com=n,min_periods=n).mean()
    return df["ATR"]


In [36]:
def renko_DF(DF, hourly_df):
    "function to convert ohlc data into renko bricks"
    df = DF.copy()
    df.reset_index(inplace=True)
    df.drop("Close",axis=1,inplace=True)
    df.columns = ["date","open","high","low","close","volume"]
    df2 = Renko(df)
    df2.brick_size = 3*round(ATR(hourly_df,120).iloc[-1],0)
    renko_df = df2.get_ohlc_data() #if using older version of the library please use get_bricks() instead
    return renko_df

for ticker in ohlcv_data:
    renko_data[ticker] = renko_DF(ohlcv_data[ticker],hour_data[ticker])

                                 Open        High  ...   Adj Close  Volume
Datetime                                           ...                    
2023-08-09 09:15:00+05:30  452.049988  452.049988  ...  451.350006       0
2023-08-09 09:20:00+05:30  451.250000  451.399994  ...  450.350006  209104
2023-08-09 09:25:00+05:30  450.299988  450.899994  ...  450.299988   83464
2023-08-09 09:30:00+05:30  450.399994  450.399994  ...  449.899994  184594
2023-08-09 09:35:00+05:30  449.899994  449.950012  ...  449.899994   81069
...                               ...         ...  ...         ...     ...
2023-09-08 15:05:00+05:30  443.149994  443.450012  ...  443.000000  403938
2023-09-08 15:10:00+05:30  443.000000  443.000000  ...  442.500000  323316
2023-09-08 15:15:00+05:30  442.450012  442.750000  ...  442.399994  574514
2023-09-08 15:20:00+05:30  442.399994  442.750000  ...  442.700012  226197
2023-09-08 15:25:00+05:30  442.649994  442.750000  ...  442.649994  232928

[1650 rows x 6 columns]


In [37]:
renko_data

{'ITC.NS':                         date   open   high    low  close  uptrend
 0  2023-08-09 09:15:00+05:30  447.0  450.0  447.0  450.0     True
 1  2023-08-09 14:35:00+05:30  450.0  453.0  450.0  453.0     True
 2  2023-08-09 14:40:00+05:30  453.0  456.0  453.0  456.0     True
 3  2023-08-09 15:15:00+05:30  456.0  459.0  456.0  459.0     True
 4  2023-08-10 12:25:00+05:30  456.0  456.0  453.0  453.0    False
 5  2023-08-10 14:25:00+05:30  453.0  453.0  450.0  450.0    False
 6  2023-08-11 11:35:00+05:30  450.0  450.0  447.0  447.0    False
 7  2023-08-16 09:25:00+05:30  450.0  453.0  450.0  453.0     True
 8  2023-08-17 09:15:00+05:30  450.0  450.0  447.0  447.0    False
 9  2023-08-17 09:35:00+05:30  447.0  447.0  444.0  444.0    False
 10 2023-08-17 12:50:00+05:30  444.0  444.0  441.0  441.0    False
 11 2023-08-21 14:55:00+05:30  444.0  447.0  444.0  447.0     True
 12 2023-08-22 09:30:00+05:30  447.0  450.0  447.0  450.0     True
 13 2023-08-22 11:50:00+05:30  450.0  453.0  450.0  